In [1]:
%load_ext autoreload
import numpy as np
import plotly.graph_objects as go

%autoreload 2
from zcu_tools.notebook.persistance import load_result
from zcu_tools.simulate.fluxonium import (
    calculate_system_n_oper_vs_flx,
    calculate_n_oper_vs_flx,
    calculate_energy_vs_flx,
)

In [2]:
qub_name = "Q12_2D/Q4"

server_ip = "021-zcu216"
port = 4999

# Load data

## Parameters

In [3]:
loadpath = f"../../result/{qub_name}/params.json"
_, params, _, _, _, result = load_result(loadpath)

r_f = result["dispersive"]["r_f"]
g = result["dispersive"]["g"]

# Matrix elements

In [27]:
sweep = None
spectrum_data = None

return_dim = 7
flxs = np.linspace(0.0, 0.5, 200)

In [ ]:
spectrum_data, energies = calculate_energy_vs_flx(
    params, flxs, spectrum_data=spectrum_data
)

In [ ]:
spectrum_data, elements = calculate_n_oper_vs_flx(
    params, flxs, return_dim=return_dim, spectrum_data=spectrum_data
)

In [ ]:
sweep, side_elements = calculate_system_n_oper_vs_flx(
    params, flxs, r_f, g=g, return_dim=return_dim, sweep=sweep
)

In [36]:
bypass_levels = [None, 2, 3]

In [ ]:
fig = go.Figure()

for f, t in [(0, 1), (1, 0)]:
    for bypass_level in bypass_levels:
        if bypass_level is None:
            m = np.abs(side_elements[:, f, t])
            label = f"{f}->{t}"
        else:
            m1 = np.abs(elements[:, f, bypass_level])
            m2 = np.abs(side_elements[:, bypass_level, t])
            # m = np.sqrt(m1 * m2)
            m = np.minimum(m1, m2)
            label = f"{f}->{bypass_level}->{t}"
        fig.add_trace(go.Scatter(x=flxs, y=m, mode="lines", name=label))

fig.update_layout(
    yaxis_range=[0, 0.2],
    xaxis_title="Flux",
    yaxis_title="Matrix Elements",
    showlegend=True,
)
fig.show()

In [38]:
prefix = f"../../result/{qub_name}"
fig.write_html(f"{prefix}/web/reset_element.html", include_plotlyjs="cdn")
fig.write_image(
    f"{prefix}/image/reset_element.png", format="png", width=800, height=400
)

In [ ]:
fig = go.Figure()

for f, t in [(0, 1), (1, 0)]:
    for bypass_level in bypass_levels:
        if bypass_level is None:
            energy = energies[:, t] - energies[:, f]
            fig.add_trace(
                go.Scatter(
                    x=flxs, y=np.abs(r_f + energy), mode="lines", name=f"{f}->{t} red"
                )
            )
        else:
            energy1 = energies[:, bypass_level] - energies[:, f]
            energy2 = energies[:, t] - energies[:, bypass_level]
            fig.add_trace(
                go.Scatter(
                    x=flxs, y=np.abs(energy1), mode="lines", name=f"{f}->{bypass_level}"
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=flxs,
                    y=np.abs(r_f + energy2),
                    mode="lines",
                    name=f"{bypass_level}->{t} red",
                )
            )

fig.update_layout(
    yaxis_range=[0, 7.0],
    xaxis_title="Flux",
    yaxis_title="Energy",
    showlegend=True,
)
fig.show()

In [40]:
prefix = f"../../result/{qub_name}"
fig.write_html(f"{prefix}/web/reset_energy.html", include_plotlyjs="cdn")